In [1]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import fits
from pyia import GaiaData
%matplotlib inline

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [4]:
data_file = 'data/GaiaDR2_vrad_vel.fits'
data_hdu = fits.open(data_file)
data_table = data_hdu[1].data
data_header = data_hdu[1].header
source_ids = data_table['source_id']
ras = data_table['ra']
decs = data_table['dec']

In [ ]:
sid_list = source_ids[50:].astype(str)
q_str = "Select phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag \
FROM gaiadr2.gaia_source as g \
WHERE g.source_id IN ("
for sid in sid_list:
    q_str += sid +','
q_str = q_str[:-1]
q_str += ')'
foo = GaiaData.from_query(q_str)

In [ ]:
foo